<div style="text-align:center">
    <h1>
        *** Project: Winter 2025 ***
    </h1>
</div>

---

<h2>I. Team members</h2>
<b>
    
- Minh Le Nguyen
- Liam Knapp
- Gautam Singh
- Gleb Ignatov

</b>
<br>

---

## II. Implementation

In [ ]:
!pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore, skew, kurtosis
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

### 1. Load Large Dataset

In [ ]:
large_dataset_dataframe = pd.read_csv("./Bigdataset/BreakData/part_1.csv", encoding='utf-8')

In [ ]:
print(large_dataset_dataframe.info)

### 2. Data Preprocessing

#### a. Large Dataset Size

In [ ]:
print("Number of rows inside the dataset: ", large_dataset_dataframe.shape[0])
print("Number of columns inside the dataset: ", large_dataset_dataframe.shape[1])

#### b. The Columns Names and Dtypes inside Large Dataframe

In [ ]:
print("The columns inside the large dataframe\n")
columns_list = large_dataset_dataframe.columns.to_list()
dtypes_list = large_dataset_dataframe.dtypes.to_list()
print(f"{'Column Name':<25} | {'Data Type'}")
print("-" * 40)
for col, dtype in zip(columns_list, dtypes_list):
    print(f"{col:<25} | {dtype}")

#### c. Summarize Data Explore inside Large Dataframe

In [ ]:
class summarize_explore():
    def __init__(self, processing_file_path):
        self.raw_df = pd.read_csv("./Bigdataset/BreakData/part_1.csv", encoding='utf-8')

    def print_dataframe_metadata(self):
        """
        Prints metadata information about a Pandas DataFrame.
        
        Parameters:
        raw_df (pd.DataFrame): The DataFrame for which metadata is to be printed.
        """
        print("DataFrame Metadata\n")

        print("\nSummary Statistics:\n")
        print(self.raw_df.describe(include='all'))
        print("\n================================================================\n")
        
        print("\nShape:\n", self.raw_df.shape)
        print("\n================================================================\n")
        
        print("\nMissing Values:\n")
        print(self.raw_df.isnull().sum())
        print("\n================================================================\n")
        
        print("\nList of Columns:\n")
        print(self.raw_df.columns.to_list())
        print("\n================================================================\n")

        print("\nColumns and Data Types:\n")
        print(self.raw_df.dtypes)
        print("\n================================================================\n")
                
        print("\nUnique Values per Column:\n")
        print(self.raw_df.nunique())
        print("\n================================================================\n")

        print("\nUnique Values per Column:\n")
        for col in self.raw_df.columns:
            print(self.raw_df[col].unique())
            print("\n================================================================\n")
        
        print("\nFirst 5 Rows:\n")
        print(self.raw_df.head())
        print("\n================================================================\n")


    def checking_missing_value(self, input_col_obj, mode="single"):
        if mode == "single" and isinstance(input_col_obj, str):
            df_missing_values_dict = dict()
            df_missing_values_dict[input_col_obj] = self.raw_df[input_col_obj].isnull().sum()
            missing_values_df = pd.DataFrame(
                list(df_missing_values_dict.items()),
                columns=["Column Name", "Number of missing values"]
            )
            return missing_values_df
        elif mode == "multiple" and isinstance(input_col_obj, list):
            df_missing_values_dict = dict()
            for col in input_col_obj:
                df_missing_values_dict[col] = self.raw_df[col].isnull().sum()
            
            missing_values_df = pd.DataFrame(
                list(df_missing_values_dict.items()),
                columns=["Column Name", "Number of missing values"]
            )
            return missing_values_df
        else:
            return "Please check the function again"

    def distribution_shape_analytics(self, df_input, column_input):
        df_process_dis = df_input.dropna().copy()
        # Distribution Shape
        skewness_temp = skew(df_process_dis[column_input])
        kurtosis_temp = kurtosis(df_process_dis[column_input])
        print("Distribution Shape Analytics for AverageTemperature:")
        print(f"Distribution Shape - Skewness: {round(skewness_temp,2)}, Kurtosis: {round(kurtosis_temp,2)}")
        if skewness_temp > 0:
            print("The distribution is positively skewed, indicating a longer tail on the right.")
        elif skewness_temp < 0:
            print("The distribution is negatively skewed, indicating a longer tail on the left.")
        else:
            print("The distribution is symmetric.")
        
        if kurtosis_temp > 0:
            print("The distribution has heavier tails and a sharper peak than a normal distribution (leptokurtic).")
        elif kurtosis_temp < 0:
            print("The distribution has lighter tails and a flatter peak than a normal distribution (platykurtic).")
        else:
            print("The distribution has a kurtosis similar to a normal distribution (mesokurtic).")
        print("\n")
        return skewness_temp, kurtosis_temp
    
    def fill_missing_values(self, column_name, fill_value):
        self.raw_df[column_name] = self.raw_df[column_name].fillna(fill_value)
        return self.raw_df
        
    def plot_histogram(self, df_input, column_name, default_bin=10):
        df_processed = df_input.dropna().copy()
        column_data = df_processed[column_name].dropna()
        iqr = np.percentile(column_data, 75) - np.percentile(column_data, 25)
        bin_width_fd = 2 * iqr / (len(column_data) ** (1 / 3))
        if bin_width_fd > 0:
            n_bins_fd = int(np.ceil((column_data.max() - column_data.min()) / bin_width_fd))
        else:
            n_bins_fd = default_bin

        plt.figure(figsize=(6, 4))
        sns.histplot(column_data, bins=n_bins_fd, kde=True, color="blue", alpha=0.6)
        plt.title(f'Histogram of {column_name}')
        plt.xlabel(column_name)
        plt.ylabel('Frequency')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

    def detect_outlier(self, column_input):
        df_outliers_detect = self.raw_df.dropna().copy()
        df_outliers_detect['Z_Score'] = zscore(df_outliers_detect[column_input])
        z_outliers = df_outliers_detect[abs(df_outliers_detect['Z_Score']) > 3]
        num_z_outliers = len(z_outliers)
        Q1 = df_outliers_detect[column_input].quantile(0.25)
        Q3 = df_outliers_detect[column_input].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        iqr_outliers = df_outliers_detect[(df_outliers_detect[column_input] < lower_bound) | 
                                        (df_outliers_detect[column_input] > upper_bound)]
        num_iqr_outliers = len(iqr_outliers)
        df_outliers_detect['IQR_Outlier'] = (df_outliers_detect[column_input] < lower_bound) | (df_outliers_detect[column_input] > upper_bound)
        print(f"Number of outliers detected by Z-score method: {num_z_outliers}")
        print(f"Number of outliers detected by IQR method: {num_iqr_outliers}")
        return num_z_outliers, num_iqr_outliers
    
    def scale_numerical_variables(self, df_input, column_name, method="Min-Max"):
        df_processed = df_input.dropna().copy()
        if method == "Min-Max":
            scaler = MinMaxScaler()
            df_processed[column_name] = scaler.fit_transform(df_processed[[column_name]])
            return df_processed
        elif method == "Standardization":
            scaler = StandardScaler()
            df_processed[column_name] = scaler.fit_transform(df_processed[[column_name]])
            return df_processed
        elif method == "Robust":
            # RobustScaler uses median and IQR (interquartile range) to scale and making it robust to outliers
            scaler = RobustScaler()
            df_processed[column_name] = scaler.fit_transform(df_processed[[column_name]])
            return df_processed
        elif method == "Log":
            # log(1 + x) to avoid log(0) use logarithmic transformation for normalizing skewed data
            df_processed[column_name] = np.log1p(df_processed[column_name])
            return df_processed
        else:
            raise ValueError("Invalid scaling method. Choose from ['Min-Max', 'Standardization', 'Robust', 'Log']")

    def convert_categorical_variables(self, df_input):
        df_processed = df_input['userId'].dropna().copy()
        df_input['churn'] = df_input['page'].apply(lambda x: 1 if x == "Cancellation Confirmation" else 0)
        return df_input

#### d. Print Meta Data

In [ ]:
summarize_explore_ = summarize_explore("./Bigdataset/BreakData/part_1.csv") 
df_large_dataset_raw = summarize_explore_.raw_df
column_name = df_large_dataset_raw.columns.to_list()
summarize_explore_.print_dataframe_metadata()

#### e. Check Missing Values inside Columns inside Large Dataset

In [ ]:
summarize_explore_.checking_missing_value(column_name, mode="multiple")

#### f. Define the Churn

In [ ]:
df_large_dataset_churn_processed = summarize_explore_.convert_categorical_variables(df_large_dataset_raw)

In [ ]:
visualize_large_df_explore = df_large_dataset_churn_processed.copy()
list_columns = visualize_large_df_explore.drop("userId", axis=1).select_dtypes(include=['int64', 'float64']).columns
for col in list_columns:
    summarize_explore_.plot_histogram(visualize_large_df_explore, col)

#### g, Checking Non Missing UserId and Churn Portion

In [ ]:
print("\nProportion (percentage) of each churn value:")
df_large_dataset_non_missing = df_large_dataset_churn_processed.filter(df_large_dataset_churn_processed["userId"] != "")
print(len(df_large_dataset_non_missing))
print("---------------------------------------------------------------------")
print(df_large_dataset_churn_processed['churn'].value_counts(normalize=True))
print("---------------------------------------------------------------------")
print("Count of each churn value:")
print(df_large_dataset_churn_processed['churn'].value_counts())

#### h, Remove duplicates based on 'userId'

In [ ]:
df_large_dataset_churn_processed_non_missing = df_large_dataset_churn_processed[df_large_dataset_churn_processed["userId"].notna() & (df_large_dataset_churn_processed["userId"] != "")]
df_large_dataset_churn_processed_non_missing_non_duplicated = df_large_dataset_churn_processed_non_missing.drop_duplicates()
print("Number of records after processed removed missing and duplicated", df_large_dataset_churn_processed_non_missing_non_duplicated.shape[0])

### 3. Data Exploratory Analysis

#### A, Calculate average churn rate by gender

In [ ]:
gender_stat_large_df = df_large_dataset_churn_processed_non_missing_non_duplicated[['gender', 'churn']]
avg_churn_by_gender = gender_stat_large_df.groupby('gender')['churn'].mean() * 100

print('The avg churn rate of females is:', avg_churn_by_gender.get('F', 'N/A'))
print('The avg churn rate of males is:', avg_churn_by_gender.get('M', 'N/A'))

avg_churn_by_gender.plot(kind='bar')
plt.ylabel('Average Churn Rate (%)')
plt.title('Average Churn Rate by Gender')
plt.xticks(rotation=0)
plt.show()

#### B, Calculate average churn rate by artist

In [ ]:
artist_stat_large_df = df_large_dataset_churn_processed_non_missing_non_duplicated[['artist', 'churn']].copy()
top_artists_by_churn = (
    artist_stat_large_df.groupby('artist', dropna=False)['churn']
    .sum()
    .sort_values(ascending=False)
    .head(5)
)
top_artists_by_churn

#### C, Calculate average churn rate by level

In [ ]:
level_stat_large_df = df_large_dataset_churn_processed_non_missing_non_duplicated[['level', 'churn']].copy()
level_stat_large_df['churn'] = pd.to_numeric(level_stat_large_df['churn'], errors='coerce')
churn_by_level = level_stat_large_df.groupby('level')['churn'].mean() * 100

print('Proportion of users that churned from free subscription:', churn_by_level.get('free', 'N/A'))
print('Proportion of users that churned from paid subscription:', churn_by_level.get('paid', 'N/A'))

churn_by_level.plot(kind='bar')
plt.ylabel('Average Churn Rate (%)')
plt.title('Average Churn Rate by By Subcription Type')
plt.xticks(rotation=0)
plt.show()

#### D, Number of churns per State

In [ ]:
location_stat_large_df = df_large_dataset_churn_processed_non_missing_non_duplicated[['location', 'churn']].copy()
location_stat_large_df['churn'] = pd.to_numeric(location_stat_large_df['churn'], errors='coerce')
location_stat_large_df['state'] = location_stat_large_df['location'].str.split(',').str[1].str.strip()
state_stat_large_df = location_stat_large_df.copy()
state_stat_large_df.drop(columns='location', inplace=True)
state_churn = state_stat_large_df.groupby('state')['churn'].sum().reset_index()
state_churn = state_churn[state_churn['churn'] > 0]
top_states = state_churn.sort_values(by='churn', ascending=False).reset_index().drop(columns='index').head(10)

print('Viewing top 10 states with churn:\n')
print(top_states)

top_states.plot(kind='bar', x='state', y='churn', legend=False)
plt.ylabel('Total Churn Count')
plt.title('Top 10 States by Total Churn')
plt.xticks(rotation=360)
plt.tight_layout()
plt.show()

#### E, Time-Series Analysis

##### Convert 'ts' from milliseconds to datetime

In [ ]:
large_df_processed_time = df_large_dataset_churn_processed_non_missing_non_duplicated.copy()
large_df_processed_time['ts'] = pd.to_datetime(large_df_processed_time['ts'], unit='ms')

large_df_processed_time['hour'] = large_df_processed_time['ts'].dt.hour
large_df_processed_time['day'] = large_df_processed_time['ts'].dt.day
large_df_processed_time['month'] = large_df_processed_time['ts'].dt.month
large_df_processed_time['week_day'] = large_df_processed_time['ts'].dt.weekday  

In [ ]:
def get_churn_distribution_by_column(column_name, churn_label, normalize=False):
    filtered_df = large_df_processed_time[large_df_processed_time['churn'] == churn_label]
    churn_distribution = filtered_df.groupby(column_name)['userId'].count()
    try:
        churn_distribution.index = churn_distribution.index.astype(int)
    except:
        pass
    if normalize:
        churn_distribution = churn_distribution / churn_distribution.sum() * 100
    return churn_distribution.sort_index()

In [ ]:
def plot_churn_distribution_by_time(time_column, normalize=True, figsize=(16, 4), chart_title=None, label_rotation=0):
    churn_time_df = pd.DataFrame({
        'Churned Users': get_churn_distribution_by_column(time_column, churn_label=1, normalize=normalize),
        'Active Users': get_churn_distribution_by_column(time_column, churn_label=0, normalize=normalize)
    })
    ax = churn_time_df.plot(kind='bar', figsize=figsize)
    if chart_title is None:
        chart_title = time_column
    ax.set_ylabel('Percentage of Users' if normalize else 'User Count')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=label_rotation)
    ax.set_title(f'{"Percentage" if normalize else "Count"} of Users by {chart_title}')
    return ax

##### i, Percentage of Users Churn by Hour of the Day

In [ ]:
plot_churn_distribution_by_time('hour', chart_title='Hour of the Day')
plt.show()

##### ii, Percentage of Users Churn by Day in a Month

In [ ]:
plot_churn_distribution_by_time('day', chart_title='Day In A Month')
plt.show()

##### iii, Percentage of Users Churn by Weekday

In [ ]:
plot_churn_distribution_by_time('week_day', chart_title='Weekday')
plt.show()

##### iv, Percentage of Users Churn by Month

In [ ]:
plot_churn_distribution_by_time('month', chart_title='Month')
plt.show()

### 4. Feature Engineering

In [ ]:
feature_engineering_large_df = large_df_processed_time.copy()
feature_engineering_large_df['churn'] = pd.to_numeric(feature_engineering_large_df['churn'], errors='coerce')
feature_engineering_large_df['state'] = feature_engineering_large_df['location'].str.split(',').str[1].str.strip()

##### i. Label encode ordinal columns

In [ ]:
ordinal_cols = ['level']
le = LabelEncoder()
for col in ordinal_cols:
    feature_engineering_large_df[col] = le.fit_transform(feature_engineering_large_df[col].astype(str))

##### ii. One-hot encode nominal categorical columns

In [ ]:
nominal_cols = [
    'page', 'auth', 'method', 'location', 'gender', 'state'
]

nominal_backup = feature_engineering_large_df[nominal_cols].copy()
feature_engineering_large_df[nominal_cols] = feature_engineering_large_df[nominal_cols].astype(str)
feature_engineering_large_df = pd.get_dummies(feature_engineering_large_df, columns=nominal_cols, drop_first=False)
feature_engineering_large_df = pd.concat([feature_engineering_large_df, nominal_backup], axis=1)

##### iii. Calculate the length of object columns

In [ ]:
object_columns_large_df = ['userAgent', 'lastName', 'firstName', 'artist', 'song']

for col in object_columns_large_df:
    feature_engineering_large_df[f"value_length_{col}"] = feature_engineering_large_df[col].astype(str).apply(len)

#### A, Feature LifeTime Since Registration

In [ ]:
feature_engineering_large_df['registration'] = pd.to_datetime(feature_engineering_large_df['registration'], unit='ms')
feature_engineering_large_df['lifetime'] = (feature_engineering_large_df['ts'] - feature_engineering_large_df['registration']).dt.total_seconds()

feature_1 = feature_engineering_large_df.groupby('userId')['lifetime'].max().reset_index()
feature_1['lifetime'] = feature_1['lifetime'] / (3600 * 24)

print(feature_1.describe())

#### B, Feature Total Songs Listened

In [ ]:
feature_2 = feature_engineering_large_df.groupby('userId')['song'].count().reset_index()
feature_2 = feature_2.rename(columns={'song': 'total_songs'})

print(feature_2.describe())

#### C, Feature Total Songs Liked

In [ ]:
feature_3 = feature_engineering_large_df[feature_engineering_large_df['page'] == 'Thumbs Up'].groupby('userId')['page'].count().reset_index()
feature_3 = feature_3.rename(columns={'page': 'num_thumb_up'})

print(feature_3.describe())

#### D, Feature Total Songs Disliked

In [ ]:
feature_4 = feature_engineering_large_df[feature_engineering_large_df['page'] == 'Thumbs Down'].groupby('userId')['page'].count().reset_index()
feature_4 = feature_4.rename(columns={'page': 'num_thumb_down'})

print(feature_4.describe())

#### E, Feature Playlist length: (Check for all the Add to Playlist Page)

In [ ]:
feature_5 = feature_engineering_large_df[feature_engineering_large_df['page'] == 'Add to Playlist'].groupby('userId')['page'].count().reset_index()
feature_5 = feature_5.rename(columns={'page': 'add_to_playlist'})

print(feature_5.describe())

#### F, Feature Referring friends: (Check for All the Add Friend Page)

In [ ]:
feature_6 = feature_engineering_large_df[feature_engineering_large_df['page'] == 'Add Friend'].groupby('userId')['page'].count().reset_index()
feature_6 = feature_6.rename(columns={'page': 'add_friend'})

print(feature_6.describe())

#### G, Feature Listening Longevity: ( Check for All the total listen time each user)

In [ ]:
feature_7 = feature_engineering_large_df.groupby('userId')['length'].sum().reset_index()
feature_7 = feature_7.rename(columns={'length': 'listen_time'})

print(feature_7.describe())

#### H, Feature Songs per Session: (Avange song played per Sessions Count the number user hit NextSong group by sessionId, userId and take avarange of the number of song be played corresponding to that SessionID)

In [ ]:
next_song_df = feature_engineering_large_df[feature_engineering_large_df['page'] == 'NextSong']
songs_per_session = next_song_df.groupby(['userId', 'sessionId']).size().reset_index(name='song_count')

feature_8 = songs_per_session.groupby('userId')['song_count'].mean().reset_index()
feature_8 = feature_8.rename(columns={'song_count': 'avg_songs_played'})

print(feature_8.describe())

#### I, Feature Gender

In [ ]:
feature_9 = feature_engineering_large_df[['userId', 'gender_F']].drop_duplicates()
feature_9 = feature_9.rename(columns={'gender_F': 'gender'})

feature_9['gender'] = feature_9['gender'].astype(int)

print(feature_9.describe())

#### K, Feature Number of Artists Listened: ( Count the total of Artists that each user listen to)

In [ ]:
next_song_df = feature_engineering_large_df[feature_engineering_large_df['page'] == 'NextSong']

artist_per_session = (
    next_song_df.groupby(['userId', 'sessionId'])['artist']
    .nunique()
    .reset_index(name='unique_artist_count')
)

feature_10 = (
    artist_per_session.groupby('userId')['unique_artist_count']
    .mean()
    .reset_index(name='avg_unique_artists_per_session')
)

print(feature_10.describe())

#### L, Feature Number Time User Login

In [ ]:
feature_11 = (
    feature_engineering_large_df[feature_engineering_large_df['page'] == 'Logged In']
    .groupby('userId')['page']
    .count()
    .reset_index()
    .rename(columns={'page': 'login_count'})
)

print(feature_11.describe())

#### M Feature Number Time User Logout

In [ ]:
feature_12 = (
    feature_engineering_large_df[feature_engineering_large_df['page'] == 'Logged Out']
    .groupby('userId')['page']
    .count()
    .reset_index()
    .rename(columns={'page': 'logout_count'})
)

print(feature_13.describe())

#### N, Feature Number Time User Downgrade

In [ ]:
feature_13 = (
    feature_engineering_large_df[feature_engineering_large_df['page'] == 'Downgrade']
    .groupby('userId')['page']
    .count()
    .reset_index()
    .rename(columns={'page': 'downgrade_count'})
)
print(feature_14.describe())

#### Label

In [ ]:
label_churn = feature_engineering_large_df[['userId', 'churn']].drop_duplicates().rename(columns={'churn': 'label'})
print(label_churn.describe())

In [ ]:
merged_data = feature_1 \
    .merge(feature_2, on='userId', how='outer') \
    .merge(feature_3, on='userId', how='outer') \
    .merge(feature_4, on='userId', how='outer') \
    .merge(feature_5, on='userId', how='outer') \
    .merge(feature_6, on='userId', how='outer') \
    .merge(feature_7, on='userId', how='outer') \
    .merge(feature_8, on='userId', how='outer') \
    .merge(feature_9, on='userId', how='outer') \
    .merge(feature_10, on='userId', how='outer') \
    .merge(feature_11, on='userId', how='outer') \
    .merge(feature_12, on='userId', how='outer') \
    .merge(feature_13, on='userId', how='outer') \
    .merge(label_churn, on='userId', how='outer') \
    .drop(columns='userId').fillna(0)

print(merged_data.describe())
print(merged_data.count())

In [ ]:
correlation_matrix = merged_data.corr(numeric_only=True)

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', center=0, square=True, linewidths=0.5)

plt.title('Correlation Heatmap of All Features (Including Label)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()